In [1]:
import pandas as pd
import numpy as np
import sklearn
import torch
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import torch.nn as nn
from torch.utils.data import DataLoader,TensorDataset
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

In [2]:
#Seed 고정
random_seed=1
random.seed(random_seed)
torch.manual_seed(random_seed)
torch.random.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)
np.random.seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [7]:

    train_origin_data=pd.read_csv('./data/custom_contest/train.csv')
    '''
    0은 바꿀 필요가 있음 o는 숫자이므로 유지
    신고번호 = x
    신고일자 = x
    통관지세관부호 = o
    신고인부호 = 0
    수입자부호 = 0
    해외 거래처 부호 = 0
    특송업체부호 = 0 

    '''
    train_origin_data=train_origin_data.fillna('missing')
    # 데이터 확인
    print('Data 종류')
    print(train_origin_data.shape)
    print(train_origin_data.columns)
    # 쓸모없는 데이터 날리기
    train_origin_data.drop('신고번호',axis=1,inplace=True)
    train_origin_data.drop('신고일자',axis=1,inplace=True)
    train_origin_data.drop('수입자부호',axis=1,inplace=True)
    train_origin_data.drop('검사결과코드',axis=1,inplace=True)
    # train_origin_data.drop('해외거래처부호',axis=1,inplace=True)
    # train_origin_data.drop('HS10단위부호',axis=1,inplace=True)

    # target 두개 분리
    crime_target=torch.tensor(train_origin_data.pop('우범여부').to_numpy())#,dtype=torch.float)
    priority_target=torch.tensor(train_origin_data.pop('핵심적발').to_numpy())
    #numerical data 분리
    train_weight=np.log(train_origin_data.pop('신고중량(KG)').to_numpy()+1).reshape(-1,1)
    train_price=np.log(train_origin_data.pop('과세가격원화금액').to_numpy()+1).reshape(-1,1)
    train_custom_rate=train_origin_data.pop('관세율').to_numpy().reshape(-1,1)

    #replace data
    train_submit=np.load('./data/custom_contest/submit.npy',allow_pickle=True)
    train_express=np.load('./data/custom_contest/express.npy',allow_pickle=True)
    train_import=np.load('./data/custom_contest/import.npy',allow_pickle=True)
    train_company=np.load('./data/custom_contest/company.npy',allow_pickle=True)
    train_origin_data['신고인부호']=train_submit
    train_origin_data['특송부호']=train_express
    train_origin_data['수입자부호']=train_import
    train_origin_data['해외업체부호']=train_company
    # 분리 확인
    print('제거 후 데이터 종류',train_origin_data.columns)
    print(train_origin_data.tail())

    for key in train_origin_data.keys():
        enc=OneHotEncoder().fit(train_origin_data[key].to_numpy().reshape(-1,1))
        encoded_data=enc.transform(train_origin_data[key].to_numpy().reshape(-1,1))
        print(key,':',encoded_data.shape)

    # One hot encoding
    enc=OneHotEncoder(dtype=np.float32).fit(train_origin_data.to_numpy().reshape(-1,len(train_origin_data.columns)))
    train_encoded_data=enc.transform(train_origin_data.to_numpy().reshape(-1,len(train_origin_data.columns))).toarray()
    print("encoded dataset",train_encoded_data.shape)

    # numerical dataset
    train_price_tensor=torch.tensor(train_price,dtype=torch.float)
    train_weight_tensor=torch.tensor(train_weight,dtype=torch.float)
    train_custom_rate_tensor=torch.tensor(train_custom_rate,dtype=torch.float)
    # categorical dataset -> encoded data
    train_encoded_data_tensor=torch.tensor(train_encoded_data,dtype=torch.float)
    train_tensor_data=torch.cat((train_encoded_data_tensor,train_price_tensor,train_weight_tensor,train_custom_rate_tensor),dim=1)


    indices=np.arange(len(train_tensor_data))
    # train_tensor_data=torch.cat((train_price_tensor,train_weight_tensor,train_custom_rate_tensor),dim=1)
    del train_price,train_weight,train_custom_rate,train_encoded_data
    print(train_tensor_data.size())
    train_indices,test_indices=train_test_split(indices,stratify=crime_target)

    np.save('./data/custom_contest/mod_data.npy',train_tensor_data.numpy())
    np.save('./data/custom_contest/mod_crime_target.npy',crime_target)
    np.save('./data/custom_contest/mod_priority_target.npy',priority_target)
    np.save('./data/custom_contest/mod_train_index.npy',train_indices)
    np.save('./data/custom_contest/mod_test_index.npy',test_indices)
    

Data 종류
(89355, 24)
Index(['신고번호', '신고일자', '통관지세관부호', '신고인부호', '수입자부호', '해외거래처부호', '특송업체부호',
       '수입통관계획코드', '수입신고구분코드', '수입거래구분코드', '수입종류코드', '징수형태코드', '신고중량(KG)',
       '과세가격원화금액', '운송수단유형코드', '반입보세구역부호', 'HS10단위부호', '적출국가코드', '원산지국가코드',
       '관세율구분코드', '관세율', '검사결과코드', '우범여부', '핵심적발'],
      dtype='object')
제거 후 데이터 종류 Index(['통관지세관부호', '신고인부호', '해외거래처부호', '특송업체부호', '수입통관계획코드', '수입신고구분코드',
       '수입거래구분코드', '수입종류코드', '징수형태코드', '운송수단유형코드', '반입보세구역부호', 'HS10단위부호',
       '적출국가코드', '원산지국가코드', '관세율구분코드', '특송부호', '수입자부호', '해외업체부호'],
      dtype='object')
       통관지세관부호    신고인부호  해외거래처부호   특송업체부호 수입통관계획코드 수입신고구분코드  수입거래구분코드  수입종류코드  \
89350       10  Missing  missing   PR5UFJ        C        B        11      21   
89351       41  Missing  missing  missing        E        E        15      11   
89352       40    7Q31W  QW3LA8B  missing        C        B        29      21   
89353       40  Missing  TJW57CJ   O04TIW        F        B        15      21   
89354       30  Missing  0UOB

In [88]:
train_origin_data=pd.read_csv('./data/custom_contest/train.csv')
test_origin_data=pd.read_csv('./data/custom_contest/test.csv')
'''
0은 바꿀 필요가 있음 o는 숫자이므로 유지
신고번호 = x
신고일자 = x
통관지세관부호 = o
신고인부호 = 0
수입자부호 = 0
해외 거래처 부호 = 0
특송업체부호 = 0 
'''

train_origin_data=train_origin_data.fillna('missing')

# 데이터 확인
print('Data 종류')
print(train_origin_data.shape)
print(train_origin_data.columns)
# 쓸모없는 데이터 날리기
train_origin_data.drop('신고번호',axis=1,inplace=True)
train_origin_data.drop('신고일자',axis=1,inplace=True)
train_origin_data.drop('수입자부호',axis=1,inplace=True)
# train_origin_data.drop('특송업체부호',axis=1,inplace=True)
train_origin_data.drop('검사결과코드',axis=1,inplace=True)
train_origin_data.drop('해외거래처부호',axis=1,inplace=True)
train_origin_data.drop('HS10단위부호',axis=1,inplace=True)
# target 두개 분리
crime_target=torch.tensor(train_origin_data.pop('우범여부').to_numpy())#,dtype=torch.float)
priority_target=torch.tensor(train_origin_data.pop('핵심적발').to_numpy())
#numerical data 분리
epsilon=1e-7
scaler=MinMaxScaler()
train_weight=scaler.fit_transform(np.log(train_origin_data.pop('신고중량(KG)').to_numpy()+epsilon).reshape(-1,1))
train_price=scaler.fit_transform(np.log(train_origin_data.pop('과세가격원화금액').to_numpy()+epsilon).reshape(-1,1))
train_custom_rate=scaler.fit_transform(train_origin_data.pop('관세율').to_numpy().reshape(-1,1))
# 분리 확인
print('제거 후 데이터 종류\n',train_origin_data.columns)
print(train_origin_data.tail())


Data 종류
(89355, 24)
Index(['신고번호', '신고일자', '통관지세관부호', '신고인부호', '수입자부호', '해외거래처부호', '특송업체부호',
       '수입통관계획코드', '수입신고구분코드', '수입거래구분코드', '수입종류코드', '징수형태코드', '신고중량(KG)',
       '과세가격원화금액', '운송수단유형코드', '반입보세구역부호', 'HS10단위부호', '적출국가코드', '원산지국가코드',
       '관세율구분코드', '관세율', '검사결과코드', '우범여부', '핵심적발'],
      dtype='object')
제거 후 데이터 종류
 Index(['통관지세관부호', '신고인부호', '특송업체부호', '수입통관계획코드', '수입신고구분코드', '수입거래구분코드',
       '수입종류코드', '징수형태코드', '운송수단유형코드', '반입보세구역부호', '적출국가코드', '원산지국가코드',
       '관세율구분코드'],
      dtype='object')
       통관지세관부호  신고인부호   특송업체부호 수입통관계획코드 수입신고구분코드  수입거래구분코드  수입종류코드  징수형태코드  \
89350       10  M9SYU   PR5UFJ        C        B        11      21      11   
89351       41  T7VQN  missing        E        E        15      11      11   
89352       40  7Q31W  missing        C        B        29      21      11   
89353       40  UJ0JR   O04TIW        F        B        15      21      14   
89354       30  4TUUB  missing        Z        B        15      21      11   

       운송수단유형코

In [89]:
for key in train_origin_data.keys():
    enc=OneHotEncoder().fit(train_origin_data[key].to_numpy().reshape(-1,1))
    encoded_data=enc.transform(train_origin_data[key].to_numpy().reshape(-1,1))
    print(key,':',encoded_data.shape)

통관지세관부호 : (89355, 40)
신고인부호 : (89355, 965)
특송업체부호 : (89355, 81)
수입통관계획코드 : (89355, 7)
수입신고구분코드 : (89355, 4)
수입거래구분코드 : (89355, 25)
수입종류코드 : (89355, 10)
징수형태코드 : (89355, 9)
운송수단유형코드 : (89355, 6)
반입보세구역부호 : (89355, 568)
적출국가코드 : (89355, 89)
원산지국가코드 : (89355, 94)
관세율구분코드 : (89355, 35)


In [90]:

# One hot encoding
enc=OneHotEncoder(dtype=np.float32).fit(train_origin_data.to_numpy().reshape(-1,len(train_origin_data.columns)))
train_encoded_data=enc.transform(train_origin_data.to_numpy().reshape(-1,len(train_origin_data.columns))).toarray()
print("encoded dataset",train_encoded_data.shape)

# concat dataset
train_price_tensor=torch.tensor(train_price,dtype=torch.float)
train_weight_tensor=torch.tensor(train_weight,dtype=torch.float)
train_custom_rate_tensor=torch.tensor(train_custom_rate,dtype=torch.float)
train_encoded_data_tensor=torch.tensor(train_encoded_data,dtype=torch.float)
train_tensor_data=torch.cat((train_encoded_data_tensor,train_price_tensor,train_weight_tensor,train_custom_rate_tensor),dim=1)
# train_tensor_data=torch.cat((train_price_tensor,train_weight_tensor,train_custom_rate_tensor),dim=1)
del train_price,train_weight,train_custom_rate,train_encoded_data
print(train_tensor_data.size())


encoded dataset (89355, 1933)
torch.Size([89355, 1936])


In [91]:
#data 자르기
batch_size=128
test_split_rate=0.2
indices=np.arange(len(train_tensor_data))
crime_dataset=TensorDataset(train_tensor_data,crime_target)
priority_dataset=TensorDataset(train_tensor_data,priority_target.float())
# x,y train_index,test_index 보내기
# model 구조
train_indices,test_indices=train_test_split(indices,stratify=crime_target)
np.save('./data/custom_contest/mod_data.npy',train_tensor_data.numpy())
np.save('./data/custom_contest/mod_crime_target.npy',crime_target)
np.save('./data/custom_contest/mod_priority_target.npy',priority_target)
np.save('./data/custom_contest/mod_train_index.npy',train_indices)
np.save('./data/custom_contest/mod_test_index.npy',test_indices)
#crime
crime_train_dataset=Subset(crime_dataset,train_indices)
crime_test_dataset=Subset(crime_dataset,test_indices)
crime_train_data_loader=DataLoader(crime_train_dataset,batch_size=batch_size,shuffle=True)
crime_test_data_loader=DataLoader(crime_test_dataset,batch_size=batch_size,shuffle=False)
#priority
priority_train_dataset=Subset(priority_dataset,train_indices)
priority_test_dataset=Subset(priority_dataset,test_indices)
priority_train_data_loader=DataLoader(priority_train_dataset,batch_size=batch_size,shuffle=True,)
priority_test_data_loader=DataLoader(priority_test_dataset,batch_size=batch_size,shuffle=False)


In [83]:

crime_model=nn.Sequential(nn.Linear(train_tensor_data.shape[1],5000),
    nn.BatchNorm1d(5000),
    nn.ReLU(),
    nn.Linear(5000,1000),
    nn.BatchNorm1d(1000),
    nn.ReLU(),
    nn.Linear(1000,500),
    nn.BatchNorm1d(500),
    nn.ReLU(),
    nn.Linear(500,2),
    # nn.Sigmoid()
    )
# criterion=torch.nn.BCELoss()
crime_criterion=torch.nn.CrossEntropyLoss()
crime_optimizer=torch.optim.Adam(crime_model.parameters(),lr=1e-2,weight_decay=1e-4)
epochs=100

def calc_score(outputs,targets,score_dict):
  # this_correct=torch.eq(torch.max(outputs,dim=1)[1],targets).sum()
  bsz=targets.size(0)
  prediction=torch.max(outputs,dim=1)[1]
  TP = (targets * prediction).sum().to(torch.float)
  TN = ((1 - targets) * (1 - prediction)).sum().to(torch.float)
  FP = ((1 - targets) * prediction).sum().to(torch.float)
  FN = (targets * (1 - prediction)).sum().to(torch.float)
  epsilon=1e-7
  score_dict['total']+=bsz
  score_dict['TP']+=TP.item()
  score_dict['FP']+=FP.item()
  score_dict['FN']+=FN.item()
  score_dict['TN']+=TN.item()
  score_dict['accuracy'] = ((score_dict['TP']+score_dict['TN'])/score_dict['total']*100.0)
  score_dict['precision'] = (score_dict['TP']/(score_dict['TP']+score_dict['FP']+epsilon)*100.0)
  score_dict['recall'] = (score_dict['TP']/(score_dict['TP']+score_dict['FN']+epsilon)*100.0)
    
  score_dict['f1score']=(2*score_dict['precision']*score_dict['recall'])/(score_dict['precision']+score_dict['recall']+epsilon)
  
  return score_dict

def train(epoch,train_data_loader,model,optimizer,criterion):
    model.train()
    train_loss=0.0
    score_dict={
      'TP':0.0,
      'TN':0.0,
      'FP':0.0,
      'FN':0.0,
      'accuracy':0.0,
      'total':0.0,
      'precision':0.0,
      'recall':0.0,
      'f1score':0.0,
      'loss':0.0,
    }

    for batch_idx, (data,targets) in enumerate(train_data_loader):
        data,targets=data.cuda(),targets.cuda()
        outputs=model(data)
        loss=criterion(outputs,targets)#.view(-1,1))
        score_dict=calc_score(outputs,targets,score_dict)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()
        if batch_idx%50==1:
            # print(outputs)
            # print(torch.eq(torch.ge(outputs,0.5).float(),targets.view(-1,1)).view(-1))
            print('\r{}epoch {}/{}, [Acc] {:.2f} [Loss] {:.5f}'.format(epoch,int(score_dict['total']),
            len(train_data_loader.dataset),score_dict['accuracy'],train_loss/(batch_idx+1)),end='')
    score_dict['loss']=train_loss/(batch_idx+1)
    return score_dict

def eval(epoch,test_data_loader,model,optimizer,criterion):
  
    score_dict={
      'TP':0.0,
      'TN':0.0,
      'FP':0.0,
      'FN':0.0,
      'accuracy':0.0,
      'total':0.0,
      'precision':0.0,
      'recall':0.0,
      'f1score':0.0,
    }

    model.eval()
    eval_loss=0.0

    with torch.no_grad():
      for batch_idx,(data,targets) in enumerate(test_data_loader):
          data,targets=data.cuda(),targets.cuda()
          outputs=model(data)
          score_dict=calc_score(outputs,targets,score_dict)
          loss=criterion(outputs,targets)

          eval_loss +=loss.item()
    score_dict['loss']=eval_loss/(batch_idx+1)
    return score_dict


crime_model.cuda()
for epoch in range(1,epochs+1):
    print('='*30)
    score_dict=train(epoch,crime_train_data_loader,crime_model,crime_optimizer,crime_criterion)
    print('\n[{}epoch Train] [loss] {:.5f} [acc] {:.2f} [precision] {:.2f} [recall] {:.2f} [f1score] {:.2f}'.format(epoch,score_dict['loss'], score_dict['accuracy'],score_dict['precision'],score_dict['recall'],score_dict['f1score']))
    score_dict=eval(epoch,crime_test_data_loader,crime_model,crime_optimizer,crime_criterion)
    print('\n[{}epoch Test] [loss] {:.5f} [acc] {:.2f} [precision] {:.2f} [recall] {:.2f} [f1score] {:.2f}'.format(epoch,score_dict['loss'], score_dict['accuracy'],score_dict['precision'],score_dict['recall'],score_dict['f1score']))
    print('='*30)

 [precision] 59.40 [recall] 31.91 [f1score] 41.51
15epoch 64256/67016, [Acc] 79.67 [Loss] 0.43338
[15epoch Train] [loss] 0.43449 [acc] 79.60 [precision] 60.74 [recall] 32.09 [f1score] 42.00

[15epoch Test] [loss] 0.44116 [acc] 79.35 [precision] 60.88 [recall] 28.74 [f1score] 39.04
16epoch 64256/67016, [Acc] 79.54 [Loss] 0.43448
[16epoch Train] [loss] 0.43495 [acc] 79.52 [precision] 60.29 [recall] 32.23 [f1score] 42.00

[16epoch Test] [loss] 0.44060 [acc] 79.14 [precision] 62.13 [recall] 23.91 [f1score] 34.53
17epoch 64256/67016, [Acc] 79.58 [Loss] 0.43385
[17epoch Train] [loss] 0.43480 [acc] 79.51 [precision] 60.43 [recall] 31.74 [f1score] 41.62

[17epoch Test] [loss] 0.44196 [acc] 79.28 [precision] 59.77 [recall] 30.41 [f1score] 40.31
18epoch 64256/67016, [Acc] 79.72 [Loss] 0.43427
[18epoch Train] [loss] 0.43432 [acc] 79.72 [precision] 61.34 [recall] 32.14 [f1score] 42.18

[18epoch Test] [loss] 0.44060 [acc] 79.18 [precision] 59.89 [recall] 28.85 [f1score] 38.94
19epoch 64256/67016, [

In [24]:
priority_model=nn.Sequential(nn.Linear(train_tensor_data.shape[1],5000),
    nn.BatchNorm1d(5000),
    nn.ReLU(),
    nn.Linear(5000,1000),
    nn.BatchNorm1d(1000),
    nn.ReLU(),
    nn.Linear(1000,100),
    nn.BatchNorm1d(100),
    nn.ReLU(),
    nn.Linear(100,1)
)
priority_criterion=torch.nn.MSELoss()
priority_optimizer=torch.optim.Adam(priority_model.parameters(),lr=1e-3,weight_decay=1e-4)
epochs=100

In [20]:
def train(epoch,train_data_loader,model,optimizer,criterion):
  model.train()
  total=0
  correct=0.0
  train_loss=0.0
  for batch_idx, (data,targets) in enumerate(train_data_loader):
      data,targets=data.cuda(),targets.cuda()
      outputs=model(data)
      loss=criterion(torch.clip(outputs,0,2),targets.view(-1,1))

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      # correct+=torch.eq(torch.ge(outputs,0.5).float(),targets.view(-1,1)).sum()
      correct+=torch.eq(torch.round(outputs),targets.view(-1,1)).sum()
      total += targets.size(0)
      train_loss+=loss.item()
      if batch_idx%50==1:
          # print(outputs)
          # print(torch.eq(torch.ge(outputs,0.5).float(),targets.view(-1,1)).view(-1))
          print('\r{}epoch {}/{}, Accurcay: {:.2f} Loss:{:.5f}'.format(epoch,total,len(train_data_loader.dataset),correct/total*100.0,train_loss/(batch_idx+1)),end='')

def eval(epoch,test_data_loader,model,optimizer,criterion):
    model.eval()
    eval_loss=0.0
    correct=0.0
    total=0.0
    with torch.no_grad():
      for batch_idx,(data,targets) in enumerate(test_data_loader):
          data,targets=data.cuda(),targets.cuda()
          outputs=model(data)
          loss=criterion(outputs,targets.view(-1,1))
          # correct+=torch.eq(torch.ge(outputs,0.5).float(),targets.view(-1,1)).sum()
          correct+=torch.eq(torch.round(outputs),targets.view(-1,1)).sum()
          total += targets.size(0)
          eval_loss +=loss.item()

    # print('\n Eval: {} epoch, Accuracy: {:.2f}, Loss: {}'.format(epoch,correct/total*100.0,eval_loss/(batch_idx+1)))


priority_model.cuda()
for epoch in range(1,epochs+1):
  score_dict=train(epoch,priority_train_data_loader,priority_model,priority_optimizer,priority_criterion)
  print('[{}epoch Train]'.format(epoch),score_dict)
  scoer_dict=eval(epoch,priority_test_data_loader,priority_model,priority_optimizer,priority_criterion)
  print('[{}epoch Eval]'.format(epoch),score_dict)


NameError: name 'priority_model' is not defined